In [1]:
from polars.dependencies import torch

from model import *

In [2]:
net = EmbModel(vocab_size=256*4)

In [3]:
input_data = torch.randint(0, 256*4, (1228, 32), dtype=torch.long).to('cuda')  # shape: [3, 16]

In [4]:
input_data[0]

tensor([403, 488, 777, 693, 218, 584, 116,  95, 993, 654, 465, 859, 338, 563,
        949,  37, 958, 671, 733, 292, 263, 975, 853, 398, 342, 123, 873, 625,
        479, 750,  12, 152], device='cuda:0')

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
net.to('cuda')

EmbModel(
  (encoder_embed): Embedding(1024, 128)
  (encoder): Sequential(
    (0): Linear(in_features=4096, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=128, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): ReLU()
    (6): Linear(in_features=1024, out_features=32768, bias=True)
  )
)

In [7]:
for epoch in range(25):
    for batch in input_data:
        net.train()
        # batch: [batch_size, 16]
        optimizer.zero_grad()
        output = net(batch.unsqueeze(0))  # [batch_size, 16, vocab_size]

        # Вычисляем потерю для каждого токена в последовательности
        loss = criterion(output.view(-1, 256*4), batch.unsqueeze(0).view(-1))

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch}, Loss: {loss.item()}")

Epoch 0, Loss: 6.544066429138184
Epoch 1, Loss: 6.54044246673584
Epoch 2, Loss: 6.537015438079834
Epoch 3, Loss: 6.533761978149414
Epoch 4, Loss: 6.53067684173584
Epoch 5, Loss: 6.527747631072998
Epoch 6, Loss: 6.524967193603516
Epoch 7, Loss: 6.522325038909912
Epoch 8, Loss: 6.519815921783447
Epoch 9, Loss: 6.517430782318115
Epoch 10, Loss: 6.515161514282227
Epoch 11, Loss: 6.513003826141357
Epoch 12, Loss: 6.510951995849609
Epoch 13, Loss: 6.508997440338135
Epoch 14, Loss: 6.507136344909668
Epoch 15, Loss: 6.5053629875183105
Epoch 16, Loss: 6.503673553466797
Epoch 17, Loss: 6.502063274383545
Epoch 18, Loss: 6.5005292892456055
Epoch 19, Loss: 6.499063968658447
Epoch 20, Loss: 6.4976677894592285
Epoch 21, Loss: 6.496335029602051
Epoch 22, Loss: 6.495060920715332
Epoch 23, Loss: 6.493847370147705
Epoch 24, Loss: 6.492687225341797


In [8]:
net.eval()

EmbModel(
  (encoder_embed): Embedding(1024, 128)
  (encoder): Sequential(
    (0): Linear(in_features=4096, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=128, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): ReLU()
    (6): Linear(in_features=1024, out_features=32768, bias=True)
  )
)

In [9]:
torch.argmax(net(input_data[0].unsqueeze(0).to('cuda')).to('cpu'), dim=-1)

tensor([[ 318,  565,  533,  342,  565,  182,  789,  262,  802,  595, 1021,  569,
          239,  440,  807,   64,  999,  442,  554,  342,  431,  493,  119,  979,
          939,  651,  656,   11,  501,  500,  835,  552]])

In [10]:
input_data[0].unsqueeze(0)

tensor([[403, 488, 777, 693, 218, 584, 116,  95, 993, 654, 465, 859, 338, 563,
         949,  37, 958, 671, 733, 292, 263, 975, 853, 398, 342, 123, 873, 625,
         479, 750,  12, 152]], device='cuda:0')